# Projecto Zipline

In [1]:
import datetime as dt
import pandas as pd
import pandas_datareader.data as web
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
from zipline.api import order_target, record, symbol
%matplotlib inline

In [6]:
start = dt.datetime(2016,12,1)
end = dt.datetime(2016,12,15)


In [7]:
df = web.DataReader('TSLA','google',start,end)

In [8]:
df.head()

,Open,High,Low,Close,Volume
Date,,,,,
2016-12-01,188.25,188.53,181.00,181.88,5126401
2016-12-02,182.88,184.88,180.00,181.47,4042324
2016-12-05,182.51,188.89,182.51,186.80,4072239
2016-12-06,185.52,186.58,182.68,185.85,3391622
2016-12-07,186.15,193.40,185.00,193.15,5461851


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 11 entries, 2016-12-01 to 2016-12-15
Data columns (total 5 columns):
Open      11 non-null float64
High      11 non-null float64
Low       11 non-null float64
Close     11 non-null float64
Volume    11 non-null int64
dtypes: float64(4), int64(1)
memory usage: 528.0 bytes


In [ ]:
plt.plot(df['Open'])

In [ ]:
plt.show()